In [1]:
import pandas as pd
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
import wandb
import wandb.sklearn
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
import pickle

In [2]:
df = pd.read_csv('bankloan.csv')

df.describe()

,age,ed,employ,address,income,debtinc,creddebt,othdebt,default
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,35.387200,1.730600,8.703800,8.154000,47.676980,10.075984,1.628071,3.218582,0.251200
std,7.948403,0.960525,7.194583,6.760786,51.793192,6.659336,3.053369,6.944523,0.433747
min,20.000000,1.000000,0.000000,0.000000,12.100000,0.080000,0.005292,0.009373,0.000000
25%,29.000000,1.000000,3.000000,3.000000,24.500000,5.050000,0.396181,0.990071,0.000000
50%,35.000000,1.000000,7.000000,7.000000,34.500000,8.635000,0.906189,1.960223,0.000000
75%,41.000000,2.000000,13.000000,12.000000,54.725000,13.682500,1.859147,3.789633,1.000000
max,58.000000,5.000000,38.000000,37.000000,2461.700000,44.620000,139.580606,416.517424,1.000000


In [3]:
df.head()

,age,ed,employ,address,income,debtinc,creddebt,othdebt,default
0,41,3,17,12,35.9,11.90,0.504108,3.767992,0
1,30,1,13,8,46.7,17.88,1.352694,6.997266,0
2,40,1,15,14,61.8,10.64,3.438997,3.136523,0
3,41,1,15,14,72.0,29.67,4.165668,17.196732,0
4,57,1,7,37,25.6,15.86,1.498199,2.561961,0


In [4]:
#Factorize the categorical variables  factorize_cols: ['ed']

df['ed'] = pd.factorize(df['ed'])[0]

#standardize the numerical variables  
numerical_cols =['age', 'employ', 'address', 'income', 'debtinc', 'creddebt', 'othdebt']

scaler = StandardScaler()

df[numerical_cols] = scaler.fit_transform(df[numerical_cols])



In [5]:
X=df[df.columns.drop('default')].values

y=df["default"].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=44)

In [6]:
RUS = RandomUnderSampler(random_state=44)

#Fit the RUS
X_train_rus, y_train_rus= RUS.fit_resample(X_train, y_train)

# Check the number of records after under sampling
print(sorted(Counter(y_train_rus).items()))

[(0, 1013), (1, 1013)]


In [7]:
def train_logistic_regression(X_train, y_train, X_test, hyperparameters):
    model = LogisticRegression(**hyperparameters)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return model, y_pred

def calculate_performance_metrics(y_test, y_pred, model, X_test):
    metrics = {}
    metrics["accuracy"] = accuracy_score(y_test, y_pred)
    metrics["precision"] = precision_score(y_test, y_pred)
    metrics["recall"] = recall_score(y_test, y_pred)
    metrics["f1"] = f1_score(y_test, y_pred)
    metrics["roc_auc"] = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
    
    conf_matrix = confusion_matrix(y_test, y_pred)
    TN, FP, FN, TP = conf_matrix.ravel()
    
    metrics["ppv"] = TP / (TP + FP)
    metrics["npv"] = TN / (TN + FN)
    metrics["specificity"] = TN / (TN + FP)
    
    return metrics

def log_to_wandb(metrics, model, X_train, X_test, y_train, y_test):
    # Log metrics
    wandb.log(metrics)
    
    # Save and log the model
    os.makedirs('models', exist_ok=True)
    model_path = "models/log_model.pkl"
    with open(model_path, "wb") as f:
        pickle.dump(model, f)
    
    artifact = wandb.Artifact("log_model", type="model")
    artifact.add_file(model_path)
    wandb.log_artifact(artifact)
    
    # Save and log the data
    os.makedirs('data', exist_ok=True)
    datasets = {"training_data": pd.DataFrame(X_train), "training_labels": pd.DataFrame(y_train), 
                "test_data": pd.DataFrame(X_test), "test_labels": pd.DataFrame(y_test)}
    
    for name, df in datasets.items():
        df.to_csv(f'data/{name}.csv', index=False)
    
    artifact = wandb.Artifact('train_val_sets', type='dataset', metadata={"Source": "CaseStudy_training_data.xlsx"})
    artifact.add_dir('data')
    wandb.log_artifact(artifact)
    
    
    # Plot plots to Weights & Biases
    label_names = ["Not-Defaulted", "Defaulted"]
    y_pred_proba = model.predict_proba(X_test)
    
    wandb.sklearn.plot_class_proportions(y_train, y_test, label_names)
    wandb.sklearn.plot_summary_metrics(model, X_train, y_train, X_test, y_test)
    wandb.sklearn.plot_roc(y_test, y_pred_proba, labels=label_names)
    wandb.sklearn.plot_precision_recall(y_test, y_pred_proba, labels=label_names)
    wandb.sklearn.plot_confusion_matrix(y_test, y_pred_proba.argmax(axis=1), labels=label_names)
    

In [8]:
# Random Search Hyperparameters
sweep_config = {
    'method': 'random',
    'metric': {
      'name': 'recall', # 
      'goal': 'maximize'
    },
    'parameters': {
        'C': {
            'values': [0.1, 1, 10]
        },
        'max_iter': {
            'values': [100, 200, 300]
        },
        'penalty': {
            'values': ['l1', 'l2']
        },
        'solver': {
            'values': ['liblinear', 'saga']
        },
        'class_weight': {
            'values': ['balanced', None]
        }
    }
}

In [9]:


# Grid Search Hyperparameters
sweep_config = {
    'method': 'grid',
    'metric': {
      'name': 'recall',
      'goal': 'maximize'
    },
    'parameters': {
        'C': {
            'values': [0.1, 1, 10]
        },
        'max_iter': {
            'values': [100, 200, 300]
        },
        'penalty': {
            'values': ['l1', 'l2']
        },
        'solver': {
            'values': ['liblinear', 'saga']
        },
        'class_weight': {
            'values': ['balanced', None]
        }
    }
}



In [10]:
# Bayesian Search Hyperparameters
sweep_config = {
    'method': 'bayes',
    'metric': {
      'name': 'recall',
      'goal': 'maximize'
    },
    'parameters': {
        'C': {
            'distribution': 'uniform',
            'min': 0.1,
            'max': 10
        },
        'max_iter': {
            'distribution': 'int_uniform',
            'min': 100,
            'max': 300
        },
        'penalty': {
            'values': ['l1', 'l2']
        },
        'solver': {
            'values': ['liblinear', 'saga']
        },
        'class_weight': {
            'values': ['balanced', None]
        }
    }
}

In [11]:
def sweep():
    # Initialize wandb
    run = wandb.init()
    
    # Get hyperparameters from wandb
    hyperparameters = run.config
    
    # Train the logistic regression model
    model, y_pred = train_logistic_regression(X_train_rus, y_train_rus, X_test, hyperparameters)
    
    # Calculate performance metrics
    metrics = calculate_performance_metrics(y_test, y_pred, model, X_test)
    
    # Log to wandb
    log_to_wandb(metrics, model, X_train, X_test, y_train, y_test)
    
    # Finish the wandb run
    run.finish()

    

In [12]:
sweep_id = wandb.sweep(sweep=sweep_config, project='bankloan_sweep')
wandb.agent(sweep_id, function=sweep, count=15)  

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: mj1wox45
Sweep URL: https://wandb.ai/mozeart/bankloan_sweep/sweeps/mj1wox45


wandb: Agent Starting Run: cjjn9ek5 with config:
wandb: 	C: 7.408540810372477
wandb: 	class_weight: balanced
wandb: 	max_iter: 146
wandb: 	penalty: l2
wandb: 	solver: saga
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mozeart. Use `wandb login --relogin` to force relogin


wandb: Adding directory to artifact (.\data)... Done. 0.0s
wandb: WARNING wandb.plots.* functions are deprecated and will be removed in a future release. Please use wandb.plot.* instead.


accuracy,▁
f1,▁
npv,▁
ppv,▁
precision,▁
recall,▁
roc_auc,▁
specificity,▁
accuracy,0.75
f1,0.60443
npv,0.91489


...

wandb: Adding directory to artifact (.\data)... Done. 0.0s


accuracy,▁
f1,▁
npv,▁
ppv,▁
precision,▁
recall,▁
roc_auc,▁
specificity,▁
accuracy,0.75
f1,0.60443
npv,0.91489


wandb: Agent Starting Run: g73st5vd with config:
wandb: 	C: 8.712774120323722
wandb: 	class_weight: None
wandb: 	max_iter: 174
wandb: 	penalty: l2
wandb: 	solver: saga
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Adding directory to artifact (.\data)... Done. 0.0s


accuracy,▁
f1,▁
npv,▁
ppv,▁
precision,▁
recall,▁
roc_auc,▁
specificity,▁
accuracy,0.75
f1,0.60443
npv,0.91489
